In [13]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM

config = AutoConfig.from_pretrained('bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForMaskedLM.from_config(config)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\douli/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading configuration file https://huggingface.co/bert-base

In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd
def mask_data_loading(url):
    def stock_symbol_mask(sentense):
        pattern = r'\$[A-Z]*'
        # symbol += re.findall(pattern, sentense)
        result = re.sub(pattern, tokenizer.mask_token, sentense)

        return result

    with open(url, 'r', encoding='utf-8') as f:
        df = pd.read_json(f)
        data = df.copy()
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['sentense'] = data['body'].map(stock_symbol_mask)
        data['labels'] = data['body']
        symbols = set()
        for symbol_list in data['body'].str.findall(r'\$[A-Z]+'):
            for symbol in symbol_list:
                symbols.add(symbol)
        return data, symbols

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = mask_data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['labels', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

special_tokens_dict = {'additional_special_tokens': list(symbols)}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(29625, 768)

In [3]:
def encode(example):
    label = tokenizer(example['labels'], padding='max_length', truncation=True)
    # 101, 51, 1234, 12541, 151
    result = tokenizer(example['sentense'], padding='max_length', truncation=True)
    # 101, 103, 103, 103
    result['label_ids'] = label['input_ids']

    # masked_position = [i for i in range(len(result['input_ids'])) if result['input_ids'][i] == tokenizer.mask_token_id]
    # result['decoder_input_ids'] = label['input_ids']
    # result['labels'] = [-100 for i in label['input_ids']]
    # for i in range(len(result['labels'])):
    #     if not i in masked_position:
    #         result['labels'][i] = -100

    return result

encoded_dataset = dataset.map(encode, batched=True)

# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# print(encoded_dataset[0]['labels'])
# print(encoded_dataset[0]['label_ids'])
# print(tokenizer.ids_to_tokens)
# print(tokenizer.decode(encoded_dataset[0]['label_ids']))

100%|██████████| 15/15 [00:19<00:00,  1.29s/ba]


In [4]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# samples = encoded_dataset[:2]
# for chunk in data_collator(samples)["input_ids"]:
#     print(f"\n'>>> {tokenizer.decode(chunk)}'")

# Training

In [5]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
from transformers import TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     per_device_train_batch_size=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_dataset,
#     compute_metrics=compute_metrics,
# )

# Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=4,
    num_train_epochs=50
)

train_datset = encoded_dataset.shuffle().select(range(4000))

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_datset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: sentense. If sentense are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 50
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 50000
  1%|          | 500/50000 [02:47<4:34:52,  3.00it/s]Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json


{'loss': 0.176, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.5}


Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
  2%|▏         | 1000/50000 [05:37<4:31:07,  3.01it/s]Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json


{'loss': 0.0081, 'learning_rate': 4.9e-05, 'epoch': 1.0}


Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
  3%|▎         | 1500/50000 [08:31<4:26:50,  3.03it/s] Saving model checkpoint to test_trainer\checkpoint-1500
Configuration saved in test_trainer\checkpoint-1500\config.json


...

Model weights saved in test_trainer\checkpoint-22000\pytorch_model.bin
 45%|████▌     | 22500/50000 [2:08:23<2:31:27,  3.03it/s] Saving model checkpoint to test_trainer\checkpoint-22500
Configuration saved in test_trainer\checkpoint-22500\config.json


{'loss': 0.0002, 'learning_rate': 2.7500000000000004e-05, 'epoch': 22.5}


Model weights saved in test_trainer\checkpoint-22500\pytorch_model.bin
 46%|████▌     | 23000/50000 [2:11:17<2:29:34,  3.01it/s] Saving model checkpoint to test_trainer\checkpoint-23000
Configuration saved in test_trainer\checkpoint-23000\config.json


{'loss': 0.0004, 'learning_rate': 2.7000000000000002e-05, 'epoch': 23.0}


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\caffe2\serialize\inline_container.cc:300] . unexpected pos 317232064 vs 317231952

In [7]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)

s = '[MASK] to the sky!!!'
fill_mask(s)

[{'score': 0.9999986886978149,
  'token': 29351,
  'token_str': '$ T S L A',
  'sequence': 'to the sky!!!'},
 {'score': 2.3993442255232367e-07,
  'token': 29039,
  'token_str': '$ I M P P',
  'sequence': 'to the sky!!!'},
 {'score': 1.1034241254037624e-07,
  'token': 29448,
  'token_str': '$ M S F T',
  'sequence': 'to the sky!!!'},
 {'score': 7.492760545346755e-08,
  'token': 29010,
  'token_str': '$ P L T R',
  'sequence': 'to the sky!!!'},
 {'score': 7.289293790790907e-08,
  'token': 29250,
  'token_str': '$ S P Y',
  'sequence': 'to the sky!!!'}]

In [ ]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())